In [5]:
# where is it ?

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import TargetEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

In [6]:
train_df_full = pd.read_csv('train.csv')#'/kaggle/input/playground-series-s4e9/train.csv')
test_df_full = pd.read_csv('test.csv')#'/kaggle/input/playground-series-s4e9/test.csv')

In [10]:
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import Ridge, HuberRegressor, LogisticRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.ensemble import StackingRegressor

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

class CustomTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, stats=['mean'], fill_value=None):
        """
        stats: list of statistics for encoding, e.g., ['mean', 'median', 'mode']
        fill_value: value to fill NaNs if categories are missing in the test set (defaults to global mean)
        """
        self.stats = stats
        self.fill_value = fill_value  # Value to fill NaNs (can be mean/median or a fixed number)
        self.target_encodings_ = {stat: {} for stat in self.stats}
        self.categorical_columns_ = None  # To store categorical column names for dropping later

    def fit(self, X, y):
        self.categorical_columns_ = X.select_dtypes(include=['object', 'category']).columns  # Track categorical columns
        # Iterate over categorical columns and encode them based on the selected statistic
        for col in self.categorical_columns_:
            for stat in self.stats:
                if stat == 'mean':
                    self.target_encodings_[stat][col] = y.groupby(X[col]).mean().to_dict()
                elif stat == 'median':
                    self.target_encodings_[stat][col] = y.groupby(X[col]).median().to_dict()
                elif stat == 'mode':
                    self.target_encodings_[stat][col] = y.groupby(X[col]).agg(lambda x: pd.Series.mode(x)[0]).to_dict()
                elif stat == 'max':
                    self.target_encodings_[stat][col] = y.groupby(X[col]).max().to_dict()
                elif stat == 'min':
                    self.target_encodings_[stat][col] = y.groupby(X[col]).min().to_dict()
                else:
                    raise ValueError(f"Unsupported statistic: {stat}")
        
        # If fill_value is None, set it to the global mean of the target
        if self.fill_value is None:
            self.fill_value = y.mean()
        
        return self

    def transform(self, X):
        X_encoded = X.copy()
        # Apply the target encodings for each statistic and concatenate as new features
        for stat in self.stats:
            for col, encodings in self.target_encodings_[stat].items():
                new_col = f"{col}_{stat}"  # New column name for each encoding
                X_encoded[new_col] = X_encoded[col].map(encodings)
                
                # Fill NaNs with the predefined fill_value
                X_encoded[new_col] = X_encoded[new_col].fillna(self.fill_value)
        
        # Add isNaN columns
        for col in self.categorical_columns_:
            X_encoded[f"{col}_isNaN"] = X_encoded[col].isna().astype(int)

        # Drop the original categorical columns
        X_encoded = X_encoded.drop(columns=self.categorical_columns_)
        
        return X_encoded


In [38]:
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np
from catboost import CatBoostClassifier


# preparing data, set tuning params
SEED = 51
DEV_FRACTION = .75
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)
N = 5

print(df_dev.shape)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']

X['mileage'] = X.pop('milage') // 100  # binning mileage

# Threshold for dividing the data into "low" and "high" price
threshold = 75000

# Binary target for classification (above or below the threshold)
y_class = (y > threshold).astype(int)
print(sum(y_class))

# Define cross-validation
kfold = RepeatedKFold(n_splits=10, n_repeats=1, random_state=SEED)

# To store the RMSE results for softmax + regression and baseline
rmse_scores = []
baseline_rmse_scores = []

# Store coefficients and intercepts for both high and low models
coefficients_low = []
coefficients_high = []
intercepts_low = []
intercepts_high = []

# Store classifier feature importances (RandomForestClassifier does not have coefficients)
classifier_feature_importances = []

# Store Ridge coefficients and intercepts for baseline
baseline_coefficients = []
baseline_intercepts = []

# Perform cross-validation
for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train_reg, y_test_reg = y.iloc[train_index], y.iloc[test_index]
    y_train_class = (y_train_reg > threshold).astype(int)

    classifier = make_pipeline(
        CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
        StandardScaler().set_output(transform='pandas'),
        CatBoostClassifier(
            iterations=100, 
            depth=6, 
            learning_rate=0.1, 
            loss_function='Logloss', 
            verbose=0,  # Suppresses detailed output
            random_state=SEED
        )
    )
    classifier.fit(X_train, y_train_class)

    # Predict probability of being in the high category
    probs = classifier.predict_proba(X_test)[:, 1]  # Probability of being in the "high" category

    # Collect feature importances from the classifier (RandomForestClassifier)
    catboost_model = classifier.named_steps['catboostclassifier']
    feature_importances = catboost_model.get_feature_importance()
    classifier_feature_importances.append(feature_importances)

    # Train two separate regression models for high and low categories
    model_low = make_pipeline(
        CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
        StandardScaler().set_output(transform='pandas'),
        Ridge(alpha=10)
    )
    model_high = make_pipeline(
        CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
        StandardScaler().set_output(transform='pandas'),
        Ridge(alpha=10)
    )

    # Split training data into low and high categories based on threshold
    X_train_low = X_train[y_train_reg <= threshold]
    y_train_low = y_train_reg[y_train_reg <= threshold]
    X_train_high = X_train[y_train_reg > threshold]
    y_train_high = y_train_reg[y_train_reg > threshold]

    # Train the low and high models
    model_low.fit(X_train_low, y_train_low)
    model_high.fit(X_train_high, y_train_high)

    # Predict on the test set for both models
    preds_low = model_low.predict(X_test)
    preds_high = model_high.predict(X_test)

    # Final prediction: weighted combination of low and high model predictions based on softmax probabilities
    final_preds = probs * preds_high + (1 - probs) * preds_low

    # Evaluate the performance using RMSE
    rmse = np.sqrt(mean_squared_error(y_test_reg, final_preds))
    rmse_scores.append(rmse)

    # Collect coefficients and intercepts for both models
    ridge_low = model_low.named_steps['ridge']
    ridge_high = model_high.named_steps['ridge']

    coefficients_low.append(ridge_low.coef_)
    intercepts_low.append(ridge_low.intercept_)

    coefficients_high.append(ridge_high.coef_)
    intercepts_high.append(ridge_high.intercept_)

    # Baseline Ridge regression model
    baseline_model = make_pipeline(
        CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
        StandardScaler().set_output(transform='pandas'),
        Ridge(alpha=10)
    )
    baseline_model.fit(X_train, y_train_reg)

    # Predict on test set using baseline model
    baseline_preds = baseline_model.predict(X_test)

    # Evaluate baseline performance using RMSE
    baseline_rmse = np.sqrt(mean_squared_error(y_test_reg, baseline_preds))
    baseline_rmse_scores.append(baseline_rmse)

    # Collect baseline Ridge coefficients and intercept
    baseline_ridge = baseline_model.named_steps['ridge']
    baseline_coefficients.append(baseline_ridge.coef_)
    baseline_intercepts.append(baseline_ridge.intercept_)

# Output the mean and standard deviation of the RMSE scores for softmax + regression and baseline
print(f'Softmax + Regression CV RMSE score: {np.mean(rmse_scores):.5f} ± {np.std(rmse_scores):.5f}')
print(f'Baseline Ridge CV RMSE score: {np.mean(baseline_rmse_scores):.5f} ± {np.std(baseline_rmse_scores):.5f}')

# Output coefficients and intercepts for low and high models
'''
print("Low Model Coefficients (mean):", np.mean(coefficients_low, axis=0))
print("Low Model Intercept (mean):", np.mean(intercepts_low))
print("High Model Coefficients (mean):", np.mean(coefficients_high, axis=0))
print("High Model Intercept (mean):", np.mean(intercepts_high))

# Output baseline coefficients and intercept
print("Baseline Ridge Coefficients (mean):", np.mean(baseline_coefficients, axis=0))
print("Baseline Ridge Intercept (mean):", np.mean(baseline_intercepts))

# Output classifier feature importances
print("Classifier Feature Importances (mean):", np.mean(classifier_feature_importances, axis=0))
'''


(141400, 13)
15344
Softmax + Regression CV RMSE score: 73537.15959 ± 9964.19392
Baseline Ridge CV RMSE score: 73618.54847 ± 9985.52258


'\nprint("Low Model Coefficients (mean):", np.mean(coefficients_low, axis=0))\nprint("Low Model Intercept (mean):", np.mean(intercepts_low))\nprint("High Model Coefficients (mean):", np.mean(coefficients_high, axis=0))\nprint("High Model Intercept (mean):", np.mean(intercepts_high))\n\n# Output baseline coefficients and intercept\nprint("Baseline Ridge Coefficients (mean):", np.mean(baseline_coefficients, axis=0))\nprint("Baseline Ridge Intercept (mean):", np.mean(baseline_intercepts))\n\n# Output classifier feature importances\nprint("Classifier Feature Importances (mean):", np.mean(classifier_feature_importances, axis=0))\n'

In [31]:
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# preparing data, set tuning params
SEED = 49
DEV_FRACTION = .4
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)
N = 5

print(df_dev.shape)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']

X['mileage'] = X.pop('milage') // 100  # binning mileage

# Threshold for dividing the data into "low" and "high" price
threshold = 75000

# Binary target for classification (above or below the threshold)
y_class = (y > threshold).astype(int)
print(sum(y_class))

# Define cross-validation
kfold = RepeatedKFold(n_splits=10, n_repeats=2, random_state=SEED)

# To store the RMSE results for softmax + regression
rmse_scores = []

# Perform cross-validation
for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train_reg, y_test_reg = y.iloc[train_index], y.iloc[test_index]
    y_train_class = (y_train_reg > threshold).astype(int)

    # Classifier to predict high/low category (Softmax)
    classifier = make_pipeline(
        #TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
        CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
        StandardScaler().set_output(transform='pandas'),
        RandomForestClassifier(random_state=SEED)
    )
    classifier.fit(X_train, y_train_class)

    # Predict probability of being in the high category
    probs = classifier.predict_proba(X_test)[:, 1]  # Probability of being in the "high" category

    # Train two separate regression models for high and low categories
    model_low = make_pipeline(
        #TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
        CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
        StandardScaler().set_output(transform='pandas'),
        Ridge(alpha=10)
    )
    model_high = make_pipeline(
        #TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
        CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
        StandardScaler().set_output(transform='pandas'),
        Ridge(alpha=10)
    )

    # Split training data into low and high categories based on threshold
    X_train_low = X_train[y_train_reg <= threshold]
    y_train_low = y_train_reg[y_train_reg <= threshold]
    X_train_high = X_train[y_train_reg > threshold]
    y_train_high = y_train_reg[y_train_reg > threshold]

    # Train the low and high models
    model_low.fit(X_train_low, y_train_low)
    model_high.fit(X_train_high, y_train_high)

    # Predict on the test set for both models
    preds_low = model_low.predict(X_test)
    preds_high = model_high.predict(X_test)

    # Final prediction: weighted combination of low and high model predictions based on softmax probabilities
    final_preds = probs * preds_high + (1 - probs) * preds_low

    # Evaluate the performance using RMSE
    rmse = np.sqrt(mean_squared_error(y_test_reg, final_preds))
    rmse_scores.append(rmse)

# Output the mean and standard deviation of the RMSE scores
print(f'Softmax + Regression CV RMSE score: {np.mean(rmse_scores):.5f} ± {np.std(rmse_scores):.5f}')

# Perform baseline cross-validation
scoring = 'neg_root_mean_squared_error'
pipeline = make_pipeline(
    TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
    StandardScaler().set_output(transform='pandas'),
    Ridge(alpha=10)
)

# Perform cross-validation for baseline
baseline_scores = -cross_val_score(pipeline, X, y, scoring=scoring, cv=kfold, n_jobs=-1)

print(f'BASELINE CV RMSE score: {np.mean(baseline_scores):.5f} ± {np.std(baseline_scores):.5f}')


(75413, 13)
8191
Softmax + Regression CV RMSE score: 70801.51773 ± 12122.30700
BASELINE CV RMSE score: 69074.71681 ± 12280.39675


In [32]:
mod = pipeline.named_steps['ridge']
print(f'Ridge Coefficients: {mod.coef_}')
print(f'Ridge Intercept (Bias): {mod.intercept_}')

mod = classifier.named_steps['ridge']
print(f'Ridge Coefficients: {mod.coef_}')
print(f'Ridge Intercept (Bias): {mod.intercept_}')

mod = model_high.named_steps['ridge']
print(f'Ridge Coefficients: {mod.coef_}')
print(f'Ridge Intercept (Bias): {mod.intercept_}')

mod = model_low.named_steps['ridge']
print(f'Ridge Coefficients: {mod.coef_}')
print(f'Ridge Intercept (Bias): {mod.intercept_}')

AttributeError: 'Ridge' object has no attribute 'coef_'

# old stuffs

In [24]:
from sklearn.linear_model import Ridge, HuberRegressor, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import numpy as np


# preparing data, set tuning params
SEED = 2
DEV_FRACTION = .2
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)
N = 5
test_frac = 0.2

print(df_dev.shape)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']

X['mileage'] = X.pop('milage')//100 # binning mileage


# Threshold for dividing the data into "low" and "high" price
threshold = y.median()

# Binary target for classification (above or below the threshold)
y_class = (y > threshold).astype(int)

# Split data for training classifier and regression models
X_train, X_test, y_train_class, y_test_class, y_train_reg, y_test_reg = train_test_split(X, y_class, y, test_size=test_frac, random_state=SEED)

# Classifier to predict high/low category (Softmax)
classifier = make_pipeline(
    TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
    StandardScaler().set_output(transform='pandas'),
    RandomForestClassifier(random_state=SEED)
)
classifier.fit(X_train, y_train_class)

# Predict probability of being in the high category
probs = classifier.predict_proba(X_test)[:, 1]  # Probability of being in the "high" category

# Train two separate regression models for high and low categories
model_low = make_pipeline(
    TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
    StandardScaler().set_output(transform='pandas'),
    Ridge(alpha=10)
)
model_high = make_pipeline(
    TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
    StandardScaler().set_output(transform='pandas'),
    Ridge(alpha=10)
)

# Split training data into low and high categories based on threshold
X_train_low = X_train[y_train_reg <= threshold]
y_train_low = y_train_reg[y_train_reg <= threshold]
X_train_high = X_train[y_train_reg > threshold]
y_train_high = y_train_reg[y_train_reg > threshold]

# Train the low and high models
model_low.fit(X_train_low, y_train_low)
model_high.fit(X_train_high, y_train_high)

# Predict on the test set for both models
preds_low = model_low.predict(X_test)
preds_high = model_high.predict(X_test)

# Final prediction: weighted combination of low and high model predictions based on softmax probabilities
final_preds = probs * preds_high + (1 - probs) * preds_low

# Evaluate the performance (you can use any metric like RMSE)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test_reg, final_preds))
print(f'Final RMSE: {rmse:.5f}')



# compare against baseline
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)
scoring = 'neg_root_mean_squared_error'
pipeline = make_pipeline(
    TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
    StandardScaler().set_output(transform='pandas'),
    Ridge(alpha=10)
)

# Perform cross-validation
scores = -cross_val_score(pipeline, X, y, scoring=scoring, cv=kfold, n_jobs=-1)

print(f'BASELINE CV RMSE score: {np.mean(scores):.5f} ± {np.std(scores):.5f}')


(37707, 13)
Final RMSE: 71445.81245


In [23]:

# preparing data, set tuning params
SEED = 2
DEV_FRACTION = .5
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)
N = 5
test_frac = 0.2

print(df_dev.shape)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']

X['mileage'] = X.pop('milage')//100 # binning mileage


# Defining models with different outlier resistance
models = {
    'Ridge': Ridge(alpha=10),
    'HuberRegressor': HuberRegressor(),
    'RANSACRegressor': RANSACRegressor()
}

# Define cross-validation
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)
scoring = 'neg_root_mean_squared_error'
encoding_stats = ['mean']
ENCODERS = [
            CustomTargetEncoder(stats=['median']),  # Use multiple encoding strategies
            TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
]
# Loop over each model
for name, model in models.items():
    print(f"Evaluating {name}...")
    for encoder in ENCODERS:
        pipeline = make_pipeline(
            encoder, 
            StandardScaler().set_output(transform='pandas'),
            model
        )

        # Perform cross-validation
        scores = -cross_val_score(pipeline, X, y, scoring=scoring, cv=kfold, n_jobs=-1)
        
        print(f'{name} CV RMSE score: {np.mean(scores):.5f} ± {np.std(scores):.5f}')

(94266, 13)
Evaluating Ridge...
Ridge CV RMSE score: 75968.10291 ± 8323.04778
Ridge CV RMSE score: 75542.65268 ± 8367.63320
Ridge CV RMSE score: 76626.62285 ± 8243.98983
Ridge CV RMSE score: 74918.82361 ± 8357.89014
Evaluating HuberRegressor...
HuberRegressor CV RMSE score: 76579.93232 ± 8307.39045
HuberRegressor CV RMSE score: 76607.68289 ± 8318.64065
HuberRegressor CV RMSE score: 76506.42278 ± 8327.45015
HuberRegressor CV RMSE score: 76170.35266 ± 8280.37124
Evaluating RANSACRegressor...
RANSACRegressor CV RMSE score: 77429.08037 ± 8091.46159
RANSACRegressor CV RMSE score: 77799.80756 ± 7974.10525
RANSACRegressor CV RMSE score: 84382.58241 ± 10407.66583
RANSACRegressor CV RMSE score: 77832.36453 ± 7956.82316


In [13]:
# Defining models with different outlier resistance + logistic regression
models = {
    'Ridge': Ridge(alpha=1.0),
    'HuberRegressor': HuberRegressor(),
    'RANSACRegressor': RANSACRegressor(),
    'LogisticRegression': LogisticRegression()  # Logistic baseline for comparison
}

# Blended model (Stacking/Blending approach)
# Blend Ridge and Huber
blended_model = StackingRegressor(
    estimators=[
        ('ridge', Ridge(alpha=1.0)),
        ('huber', HuberRegressor())
    ],
    final_estimator=Ridge()
)
models['Blended_Ridge_Huber'] = blended_model

# Define cross-validation
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)
scoring = 'neg_root_mean_squared_error'

# Loop over each model
for name, model in models.items():
    print(f"Evaluating {name}...")
    pipeline = make_pipeline(
        TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
        StandardScaler().set_output(transform='pandas'),
        model
    )

    # Perform cross-validation
    scores = -cross_val_score(pipeline, X, y, scoring=scoring, cv=kfold, n_jobs=-1)
    
    print(f'{name} CV RMSE score: {np.mean(scores):.5f} ± {np.std(scores):.5f}')

Evaluating Ridge...
Ridge CV RMSE score: 89529.24871 ± 19893.50379
Evaluating HuberRegressor...
HuberRegressor CV RMSE score: 90699.35000 ± 19751.27117
Evaluating RANSACRegressor...
RANSACRegressor CV RMSE score: 91900.05487 ± 19493.58000
Evaluating LogisticRegression...


/Users/jflyn/anaconda3/envs/kaggle/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression CV RMSE score: 119875.49289 ± 11898.71066
Evaluating Blended_Ridge_Huber...
Blended_Ridge_Huber CV RMSE score: 89586.99658 ± 19810.40579


In [14]:
# Defining models with different outlier resistance
models = {
    'Ridge': Ridge(alpha=1.0),
    'HuberRegressor': HuberRegressor(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=SEED),
    'GradientBoosting': GradientBoostingRegressor(random_state=SEED)
}

# Blended model: Combining linear and tree models
blended_model_1 = StackingRegressor(
    estimators=[
        ('ridge', Ridge(alpha=1.0)),
        ('huber', HuberRegressor()),
        ('random_forest', RandomForestRegressor(n_estimators=100, random_state=SEED))
    ],
    final_estimator=GradientBoostingRegressor(random_state=SEED)
)
models['Blended_Linear_Trees_1'] = blended_model_1

# Another blend: More focus on tree models
blended_model_2 = StackingRegressor(
    estimators=[
        ('random_forest', RandomForestRegressor(n_estimators=100, random_state=SEED)),
        ('gradient_boosting', GradientBoostingRegressor(random_state=SEED))
    ],
    final_estimator=HuberRegressor()
)
models['Blended_Trees_Only'] = blended_model_2

# Define cross-validation
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)
scoring = 'neg_root_mean_squared_error'

# Loop over each model
for name, model in models.items():
    print(f"Evaluating {name}...")
    pipeline = make_pipeline(
        TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
        StandardScaler().set_output(transform='pandas'),
        model
    )

    # Perform cross-validation
    scores = -cross_val_score(pipeline, X, y, scoring=scoring, cv=kfold, n_jobs=-1)
    
    print(f'{name} CV RMSE score: {np.mean(scores):.5f} ± {np.std(scores):.5f}')

Evaluating Ridge...
Ridge CV RMSE score: 89529.24871 ± 19893.50379
Evaluating HuberRegressor...
HuberRegressor CV RMSE score: 90699.35000 ± 19751.27117
Evaluating RandomForest...
RandomForest CV RMSE score: 93587.21386 ± 18824.94566
Evaluating GradientBoosting...
GradientBoosting CV RMSE score: 91172.40501 ± 19493.14176
Evaluating Blended_Linear_Trees_1...
Blended_Linear_Trees_1 CV RMSE score: 92391.51531 ± 20424.49250
Evaluating Blended_Trees_Only...
Blended_Trees_Only CV RMSE score: 91255.51729 ± 19634.22516


# PREDICTIONS

# old stuffs

In [40]:

# Step 2: Separate features (X) and target (y) from the full training set
X_train = train_df_full.drop(columns='price')
y_train = train_df_full['price']
X_test = test_df_full




threshold = 50000
y_train_class = (y_train > threshold).astype(int)

classifier = make_pipeline(
    CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
    StandardScaler().set_output(transform='pandas'),
    CatBoostClassifier(
        iterations=100, 
        depth=6, 
        learning_rate=0.1, 
        loss_function='Logloss', 
        verbose=0,  # Suppresses detailed output
        random_state=SEED
    )
)
classifier.fit(X_train, y_train_class)

# Predict probability of being in the high category
probs = classifier.predict_proba(X_test)[:, 1]  # Probability of being in the "high" category

# Collect feature importances from the classifier (RandomForestClassifier)
catboost_model = classifier.named_steps['catboostclassifier']
feature_importances = catboost_model.get_feature_importance()
classifier_feature_importances.append(feature_importances)

# Train two separate regression models for high and low categories
model_low = make_pipeline(
    CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
    StandardScaler().set_output(transform='pandas'),
    Ridge(alpha=10)
)
model_high = make_pipeline(
    CustomTargetEncoder(stats=['mean']),  # Use multiple encoding strategies
    StandardScaler().set_output(transform='pandas'),
    Ridge(alpha=10)
)

# Split training data into low and high categories based on threshold
X_train_low = X_train[y_train <= threshold]
y_train_low = y_train[y_train <= threshold]
X_train_high = X_train[y_train > threshold]
y_train_high = y_train[y_train > threshold]

# Train the low and high models
model_low.fit(X_train_low, y_train_low)
model_high.fit(X_train_high, y_train_high)

# Predict on the test set for both models
preds_low = model_low.predict(X_test)
preds_high = model_high.predict(X_test)

# Final prediction: weighted combination of low and high model predictions based on softmax probabilities
final_preds = probs * preds_high + (1 - probs) * preds_low




# Step 5: Create a submission DataFrame
submission = pd.DataFrame({
    'id': test_df_full['id'],  # Ensure the 'id' from the test set is preserved
    'price': final_preds       # The predicted prices
})

# Step 6: Save the submission to a CSV file
submission.to_csv('submission6_outlier2stage.csv', index=False)  # Save as CSV